In [ ]:
################################################################
# IMPORTS
# - numpy, pandas: numerical + data handling
# - pyswarm: PSO implementation
# - sklearn: scaling data
# - tensorflow: for loading DNN models
# - joblib: saving/loading preprocessing objects
# - matplotlib: plotting convergence results
# - os/json/typing: file ops, configs, type hints
################################################################
from tqdm import tqdm 
import numpy as np
import pandas as pd
from pyswarm import pso
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
import os
from typing import List, Dict, Union, Tuple, Any
import json
import warnings

warnings.filterwarnings(
    "ignore", 
    message="X does not have valid feature names, but StandardScaler was fitted with feature names",
    category=UserWarning,
    module="sklearn"
)

################################################################
# CLASS: GeneralizedMOPSO
# - Implements a generalized Multi-Objective Particle Swarm Optimization
# - Handles multiple models (objectives), scalers, bounds, constraints
# - Tracks optimization history for analysis
# - Includes early termination based on convergence criteria
################################################################

class GeneralizedMOPSO:
    def __init__(self):
        """
        Constructor initializes storage dictionaries, 
        PSO variables, and optimization tracking history.
        """
        
        # Store ML models and scalers for each objective
        self.models = {}
        self.scalers_X = {}
        self.scalers_y = {}
        
        # Parameter + objective configurations
        self.parameter_info = {}
        self.objective_targets = {}
        self.objective_constraints = {}
        self.bounds = {}
        self.discrete_mappings = {}
        
        # PSO optimization tracking
        self.best_solutions_history = [] # best parameters at each iteration
        self.best_values_history = []    # best objective values
        self.actual_value_history = []   # unscaled objective values
        self.objective_histories = {}    # per-objective performance
        self.constraint_penalty_history = []
        self.constraint_violation_history = []
        self.particle_trajectories = []
        
        # PSO parameters
        self.velocity = None
        self.best_global_position = None
        self.best_global_value = np.inf
        self.inertia_weight = 0.5
        
        # Early termination parameters
        self.convergence_threshold = 10e-6
        self.stagnation_limit = 50
        self.convergence_window = []
        self.early_terminated = False
        self.termination_reason = ""
        
    
    
    ################################################################
    # ENVIRONMENT SETUP FUNCTIONS
    ################################################################
    
    def setup_environment(self):
        """Setup the optimization environment by loading models and setting up 
        base directory, objectives, parameters, models, and constraints"""
        
        print("=== GENERALIZED MULTI-OBJECTIVE PSO OPTIMIZER ===\n")
        
        # 1. Get base path for models/scalers
        
        while True:
            self.base_path = input("Enter the directory path for loading model and scalers: ").strip()
            if os.path.isdir(self.base_path):   # checks instantly if directory exists
                print(f"\n Valid path found: {self.base_path}\n")
                break
            else:
                print(f" Path not found: {self.base_path}")
                print("Please enter a correct directory path.\n")
            
               
        # 2. Define objectives
        self._setup_objectives()
        
         # 3. Define optimization input parameters (bounds, discrete/continuous)
        self._setup_input_parameters()
        
        # 4. Load ML models and scalers (IMPORTANT)
        self._load_models_and_scalers()
        
        # 5. Define optimization targets (min/target) and constraints
        self._setup_optimization_targets()
        
        print("\n Environment setup complete!")
        
    def _setup_objectives(self):
        """Setup optimization objectives"""
        """Define number and names of optimization objectives from trained DNN json file"""
        print("\n--- OPTIMIZATION OBJECTIVES SETUP ---")
        num_objectives = int(input("Enter number of objectives to optimize: "))

        print("Note the name of objective functions should be same as used for ANN training (See .json file)")
        for i in range(num_objectives):
            obj_name = input(f"Enter name for objective {i+1}: ").strip()
            self.objective_targets[obj_name] = {}
            self.objective_constraints[obj_name] = {}
            self.objective_histories[obj_name] = []
            
    def _setup_input_parameters(self):
        """Define input parameters (continuous bounds or discrete set)"""
        
        print("\n--- INPUT PARAMETERS SETUP ---")
        num_params = int(input("Enter number of input parameters: "))
        print("Note the name of input prameters should be same as used for ANN training (See .json file)")
        for i in range(num_params):
            param_name = input(f"\nEnter name for parameter {i+1}: ").strip()
            
            # Check if continuous or discrete
            is_continuous = input(f"Is '{param_name}' continuous? (y/n): ").strip().lower() == 'y'
            
            if is_continuous:
                lower = float(input(f"Enter lower bound for {param_name}: "))
                upper = float(input(f"Enter upper bound for {param_name}: "))
                self.parameter_info[param_name] = {
                    'type': 'continuous',
                    'bounds': (lower, upper)
                }
                self.bounds[param_name] = (lower, upper)
            else:
                print(f"Enter discrete values for {param_name} (comma-separated):")
                discrete_vals_input = input().strip()
                
                # Try to parse as numbers first, then as strings
                try:
                    discrete_vals = [float(x.strip()) for x in discrete_vals_input.split(',')]
                except ValueError:
                    discrete_vals = [x.strip() for x in discrete_vals_input.split(',')]
                
                self.parameter_info[param_name] = {
                    'type': 'discrete',
                    'values': discrete_vals
                }
                
                # Create mapping for discrete values
                self.discrete_mappings[param_name] = {i: val for i, val in enumerate(discrete_vals)}
                self.bounds[param_name] = (0, len(discrete_vals) - 1)
                
    def _load_models_and_scalers(self):
        """Load pre-trained models and their scalers from files"""
        
        print("\n--- LOADING MODELS AND SCALERS ---")
        
        def load_file_by_name(base_path, partial_name, loader_func):
            for file in os.listdir(base_path):
                if partial_name.lower() in file.lower():
                    print(f"Loading: {file}")
                    return loader_func(os.path.join(base_path, file))
            raise FileNotFoundError(f"No file found containing '{partial_name}' in {base_path}")
        
        for obj_name in self.objective_targets.keys():
            try:
                # Load scalers
                self.scalers_X[obj_name] = load_file_by_name(
                    self.base_path, f"{obj_name}_scaler_X", joblib.load
                )
                self.scalers_y[obj_name] = load_file_by_name(
                    self.base_path, f"{obj_name}_scaler_y", joblib.load
                )
                
                # Load model
                self.models[obj_name] = load_file_by_name(
                    self.base_path, f"{obj_name}_trained_model", tf.keras.models.load_model
                )
                
            except FileNotFoundError as e:
                print(f"Warning: {e}")
                print(f"Please ensure files exist for objective '{obj_name}'")
                
    def _setup_optimization_targets(self):
        """Setup target values, weights, and constraints for each objective"""
        print("\n--- OPTIMIZATION TARGETS & CONSTRAINTS SETUP ---")
        
        for obj_name in self.objective_targets.keys():
            print(f"\n{'='*50}")
            print(f"Setting up '{obj_name}' objective:")
            print(f"{'='*50}")
            
            # Setup optimization target
            target_type = input("Minimize (min) or target specific value (target)? ").strip().lower()
            
            if target_type == 'target':
                target_val = float(input(f"Enter target value for {obj_name}: "))
                max_val = float(input(f"Enter maximum acceptable value for {obj_name}: "))
                self.objective_targets[obj_name] = {
                    'type': 'target',
                    'target_value': target_val,
                    'max_value': max_val
                }
            else:
                max_val = float(input(f"Enter maximum value for normalization of {obj_name}: "))
                self.objective_targets[obj_name] = {
                    'type': 'minimize',
                    'max_value': max_val
                }
            
            # Setup constraints
            self._setup_constraints_for_objective(obj_name)
    
    def _setup_constraints_for_objective(self, obj_name: str):
        """Setup constraints for a specific objective"""
        print(f"\n--- CONSTRAINTS FOR '{obj_name}' ---")
        
        has_constraints = input(f"Add constraints for {obj_name}? (y/n): ").strip().lower() == 'y'
        
        if not has_constraints:
            self.objective_constraints[obj_name] = {
                'has_constraints': False
            }
            return
        
        constraints = {
            'has_constraints': True,
            'min_constraint': None,
            'max_constraint': None,
            'penalty_weight': 1000.0  # Default high penalty weight
        }
        
        # Minimum constraint
        has_min = input(f"Set minimum limit for {obj_name}? (y/n): ").strip().lower() == 'y'
        if has_min:
            min_val = float(input(f"Enter minimum allowed value for {obj_name}: "))
            constraints['min_constraint'] = min_val
            print(f"✓ Minimum constraint set: {obj_name} >= {min_val}")
        
        # Maximum constraint
        has_max = input(f"Set maximum limit for {obj_name}? (y/n): ").strip().lower() == 'y'
        if has_max:
            max_val = float(input(f"Enter maximum allowed value for {obj_name}: "))
            constraints['max_constraint'] = max_val
            print(f"✓ Maximum constraint set: {obj_name} <= {max_val}")
        
        # Penalty weight
        custom_penalty = input(f"Use custom penalty weight? (default: 1000) (y/n): ").strip().lower() == 'y'
        if custom_penalty:
            penalty = float(input("Enter penalty weight (higher = stricter constraint): "))
            constraints['penalty_weight'] = penalty
        
        self.objective_constraints[obj_name] = constraints
        
        # Print constraint summary
        print(f"\n--- CONSTRAINT SUMMARY FOR '{obj_name}' ---")
        if constraints['min_constraint'] is not None:
            print(f"  Minimum: {constraints['min_constraint']}")
        if constraints['max_constraint'] is not None:
            print(f"  Maximum: {constraints['max_constraint']}")
        print(f"  Penalty Weight: {constraints['penalty_weight']}")
        print("-" * 40)
        
    def _check_convergence(self, current_value: float) -> bool:
        """
        Check if optimization has converged based on stagnation criteria.
        Returns True if optimization should terminate early.
        """
        # Add current value to the convergence window
        self.convergence_window.append(current_value)
        
        # Keep only the last 'stagnation_limit' values
        if len(self.convergence_window) > self.stagnation_limit:
            self.convergence_window.pop(0)
        
        # Check convergence only if we have enough values
        if len(self.convergence_window) < self.stagnation_limit:
            return False
        
        # Calculate the range of values in the convergence window
        max_val = max(self.convergence_window)
        min_val = min(self.convergence_window)
        variation = abs(max_val - min_val)
        
        # Check if variation is below threshold
        if variation <= self.convergence_threshold:
            return True
        
        return False
                
    def _convert_parameters(self, x: np.ndarray) -> Dict[str, Any]:
        """Convert optimization variables to actual parameter values"""
        x = np.array(x).flatten()
        param_values = {}
        param_names = list(self.parameter_info.keys())
        
        for i, param_name in enumerate(param_names):
            param_info = self.parameter_info[param_name]
            
            if param_info['type'] == 'continuous':
                param_values[param_name] = x[i]
            else:  # discrete
                idx = int(np.round(np.clip(x[i], 0, len(param_info['values']) - 1)))
                param_values[param_name] = param_info['values'][idx]
                
        return param_values
    
    def _prepare_model_input(self, param_values: Dict[str, Any]) -> np.ndarray:
        """Prepare input array for model prediction"""
        # Convert parameter values to numerical array
        param_names = list(self.parameter_info.keys())
        x_array = []
        
        for param_name in param_names:
            param_info = self.parameter_info[param_name]
            value = param_values[param_name]
            
            if param_info['type'] == 'discrete':
                # Find the index of the discrete value
                try:
                    idx = param_info['values'].index(value)
                    x_array.append(idx)
                except ValueError:
                    # If value not found, use closest
                    x_array.append(0)
            else:
                x_array.append(float(value))
                
        return np.array(x_array)
    
    def objective_function(self, x: np.ndarray) -> List[float]:
        """Calculate objective values for given parameters"""
        param_values = self._convert_parameters(x)
        x_model = self._prepare_model_input(param_values)
        
        objectives = []
        
        for obj_name in self.objective_targets.keys():
            # Scale input
            x_scaled = self.scalers_X[obj_name].transform(x_model.reshape(1, -1))
            
            # Predict
            prediction = self.models[obj_name].predict(x_scaled, verbose=0)[0, 0]
            
            # Inverse transform
            actual_value = self.scalers_y[obj_name].inverse_transform(
                np.array([[prediction]])
            )[0, 0]
            
            objectives.append(actual_value)
            
        return objectives
    
    def calculate_constraint_penalties(self, objectives: List[float]) -> Tuple[List[float], float]:
        """Calculate constraint penalties for each objective"""
        penalties = []
        total_penalty = 0.0
        
        for i, obj_name in enumerate(self.objective_targets.keys()):
            constraint_config = self.objective_constraints[obj_name]
            actual_value = objectives[i]
            penalty = 0.0
            
            if constraint_config['has_constraints']:
                penalty_weight = constraint_config['penalty_weight']
                
                # Check minimum constraint
                if constraint_config['min_constraint'] is not None:
                    min_val = constraint_config['min_constraint']
                    if actual_value < min_val:
                        violation = min_val - actual_value
                        penalty += penalty_weight * (violation / min_val) ** 2
                
                # Check maximum constraint
                if constraint_config['max_constraint'] is not None:
                    max_val = constraint_config['max_constraint']
                    if actual_value > max_val:
                        violation = actual_value - max_val
                        penalty += penalty_weight * (violation / max_val) ** 2
            
            penalties.append(penalty)
            total_penalty += penalty
            
        return penalties, total_penalty
    def calculate_weights(self, objectives: List[float]) -> List[float]:
        """Calculate weights for each objective based on targets"""
        weights = []
        
        for i, obj_name in enumerate(self.objective_targets.keys()):
            obj_config = self.objective_targets[obj_name]
            actual_value = objectives[i]
            
            if obj_config['type'] == 'target':
                target_val = obj_config['target_value']
                max_val = obj_config['max_value']
                weight = 1.0 - min(1.0, abs(actual_value - target_val) / max_val)
            else:  # minimize
                max_val = obj_config['max_value']
                weight = 1.0 - min(1.0, actual_value / max_val)
                
            weights.append(max(0.0, weight))  # Ensure non-negative weights
            
        return weights
    
    def check_constraint_violations(self, objectives: List[float]) -> Dict[str, Dict[str, Any]]:
        """Check and report constraint violations"""
        violations = {}
        
        for i, obj_name in enumerate(self.objective_targets.keys()):
            constraint_config = self.objective_constraints[obj_name]
            actual_value = objectives[i]
            
            violation_info = {
                'has_violation': False,
                'min_violation': None,
                'max_violation': None,
                'violation_amount': 0.0
            }
            
            if constraint_config['has_constraints']:
                # Check minimum constraint
                if constraint_config['min_constraint'] is not None:
                    min_val = constraint_config['min_constraint']
                    if actual_value < min_val:
                        violation_info['has_violation'] = True
                        violation_info['min_violation'] = min_val - actual_value
                        violation_info['violation_amount'] += violation_info['min_violation']
                
                # Check maximum constraint
                if constraint_config['max_constraint'] is not None:
                    max_val = constraint_config['max_constraint']
                    if actual_value > max_val:
                        violation_info['has_violation'] = True
                        violation_info['max_violation'] = actual_value - max_val
                        violation_info['violation_amount'] += violation_info['max_violation']
            
            violations[obj_name] = violation_info
            
        return violations
    
    def combined_objective(self, x: np.ndarray) -> float:
        """Calculate combined weighted objective with constraint penalties"""
        objectives = self.objective_function(x)
        weights = self.calculate_weights(objectives)
        
        # Calculate constraint penalties
        penalties, total_penalty = self.calculate_constraint_penalties(objectives)
        
        # Normalize and combine objectives
        combined_score = 0.0
        
        for i, obj_name in enumerate(self.objective_targets.keys()):
            # Scale the objective
            scaled_obj = self.scalers_y[obj_name].transform(
                np.array([[objectives[i]]])
            )[0, 0]
            
            combined_score += weights[i] * scaled_obj
        
        # Add constraint penalties (higher penalty = worse solution)
        combined_score += total_penalty
            
        return combined_score
    
    def update_velocity(self, velocity: float, position: float, 
                       personal_best_position: float, global_best_position: float) -> float:
        """Update particle velocity"""
        inertia_term = self.inertia_weight * velocity
        personal_best_term = 2.0 * np.random.rand() * (personal_best_position - position)
        global_best_term = 2.0 * np.random.rand() * (global_best_position - position)
        return inertia_term + personal_best_term + global_best_term
    
    def pso_objective(self, x: np.ndarray) -> float:
        """PSO objective function with tracking and constraint handling"""
        actual_objectives = self.objective_function(x)
        combined_result = self.combined_objective(x)
        weights = self.calculate_weights(actual_objectives)
        
        # Calculate constraint penalties and violations
        penalties, total_penalty = self.calculate_constraint_penalties(actual_objectives)
        violations = self.check_constraint_violations(actual_objectives)
        
        # Store results
        param_values = self._convert_parameters(x)
        self.best_solutions_history.append(param_values.copy())
        self.actual_value_history.append(actual_objectives.copy())
        self.best_values_history.append(combined_result)
        self.constraint_penalty_history.append(total_penalty)
        self.constraint_violation_history.append(violations)
        
        # Store individual objective histories
        for i, obj_name in enumerate(self.objective_targets.keys()):
            scaled_obj = self.scalers_y[obj_name].transform(
                np.array([[actual_objectives[i]]])
            )[0, 0]
            weighted_obj = weights[i] * scaled_obj
            self.objective_histories[obj_name].append(weighted_obj)
        
        # Update global best
        if combined_result < self.best_global_value:
            self.best_global_position = x.copy()
            self.best_global_value = combined_result
        
        # Check for early termination based on convergence
        if self._check_convergence(combined_result):
            self.early_terminated = True
            self.termination_reason = f"Early termination: No significant improvement for {self.stagnation_limit} consecutive iterations (threshold: {self.convergence_threshold})"
            print(f"\n {self.termination_reason}")
            print(f"   Current iteration: {len(self.best_values_history)}")
            print(f"   Convergence window values: {[f'{v:.6f}' for v in self.convergence_window]}")
            print(f"   Variation in window: {abs(max(self.convergence_window) - min(self.convergence_window)):.6f}")
            
        return combined_result
    
    def run_optimization(self, swarmsize: int = 20, maxiter: int = 3000):
        """Run the PSO optimization with early termination capability"""
        print(f"\n--- RUNNING OPTIMIZATION ---")
        print(f"Swarm size: {swarmsize}, Max iterations: {maxiter}")
        print(f"Early termination: Enabled (threshold: {self.convergence_threshold}, stagnation limit: {self.stagnation_limit})")
        
        # Prepare bounds for PSO
        param_names = list(self.parameter_info.keys())
        lower_bounds = [self.bounds[name][0] for name in param_names]
        upper_bounds = [self.bounds[name][1] for name in param_names]
        
        # Initialize PSO variables
        self.velocity = np.zeros_like(lower_bounds)
        self.best_global_position = np.zeros_like(lower_bounds)
        self.best_global_value = np.inf
        
        # Reset early termination flags
        self.early_terminated = False
        self.termination_reason = ""
        self.convergence_window = []
        
        # Custom PSO implementation with early termination
        try:
            best_solution, best_value = self._custom_pso_with_early_termination(
                lower_bounds, upper_bounds, swarmsize, maxiter
            )
        except Exception as e:
            print(f"Error during PSO optimization: {e}")
            # Fallback to regular PSO if custom implementation fails
            print("Falling back to standard PSO...")
            best_solution, best_value = pso(
                self.pso_objective,
                lower_bounds,
                upper_bounds,
                swarmsize=swarmsize,
                maxiter=maxiter
            )
        
        # Print termination information
        if self.early_terminated:
            print(f"\n Optimization terminated early!")
            print(f"   Reason: {self.termination_reason}")
            print(f"   Total iterations completed: {len(self.best_values_history)}")
            print(f"   Iterations saved: {maxiter - len(self.best_values_history)}")
        else:
            print(f"\n Optimization completed all {maxiter} iterations")
            self.termination_reason = f"Completed all {maxiter} iterations"
        
        return self._convert_parameters(best_solution), best_value

    
    def _custom_pso_with_early_termination(self, lower_bounds, upper_bounds, swarmsize, maxiter):
        """
        Custom PSO implementation with early termination capability.
        This wraps the standard PSO but adds early termination logic.
        """
        iteration_count = [0]  # Use list to allow modification in nested function
        pbar = tqdm(total=maxiter, desc="Optimization Progress", ncols=100, unit="iter")
        
        def early_termination_wrapper(x):
            """Wrapper function that adds early termination logic"""
            result = self.pso_objective(x)
            iteration_count[0] += 1
            pbar.update(1)
            
            # Check if we should terminate early
            if self.early_terminated:
                # Force termination by raising a custom exception
                pbar.close()
                raise EarlyTerminationException("Early termination condition met")
            
            return result
        
        try:
            # Run PSO with the wrapper function
            best_solution, best_value = pso(
                early_termination_wrapper,
                lower_bounds,
                upper_bounds,
                swarmsize=swarmsize,
                maxiter=maxiter
            )
        except EarlyTerminationException:
            # Early termination occurred, use the best solution found so far
            best_solution = self.best_global_position
            best_value = self.best_global_value
        
        finally:
            pbar.close()  # close bar if optimization finishes normally or errors
        return best_solution, best_value
    
    def save_results(self, output_filename: str = None):
        """Save optimization results to Excel"""
        if output_filename is None:
            objectives_str = "_".join(self.objective_targets.keys())
            output_filename = f'MOPSO_Results_{objectives_str}.xlsx'
            
        output_path = os.path.join(self.base_path, output_filename)
        
        # Prepare data for DataFrame
        data = []
        param_names = list(self.parameter_info.keys())
        obj_names = list(self.objective_targets.keys())
        
        for i in range(len(self.best_solutions_history)):
            row = {'Iteration': i + 1}
            
            # Add parameter values
            for param_name in param_names:
                row[param_name] = self.best_solutions_history[i][param_name]
            
            # Add objective values
            for j, obj_name in enumerate(obj_names):
                row[f'Actual_{obj_name}'] = self.actual_value_history[i][j]
                row[f'Normalized_Weighted_{obj_name}'] = self.objective_histories[obj_name][i]
                
                # Add constraint information
                violations = self.constraint_violation_history[i][obj_name]
                row[f'{obj_name}_Constraint_Violated'] = violations['has_violation']
                if violations['has_violation']:
                    row[f'{obj_name}_Violation_Amount'] = violations['violation_amount']
                else:
                    row[f'{obj_name}_Violation_Amount'] = 0.0
            
            row['Combined_Objective_Value'] = self.best_values_history[i]
            row['Total_Constraint_Penalty'] = self.constraint_penalty_history[i]
            row['Is_Feasible'] = self.constraint_penalty_history[i] == 0.0
            data.append(row)
        
        df = pd.DataFrame(data)
        
        # Add termination information to a separate sheet
        termination_info = pd.DataFrame([{
            'Termination_Reason': self.termination_reason,
            'Early_Terminated': self.early_terminated,
            'Total_Iterations': len(self.best_values_history),
            'Convergence_Threshold': self.convergence_threshold,
            'Stagnation_Limit': self.stagnation_limit,
            'Final_Best_Value': self.best_global_value if hasattr(self, 'best_global_value') else 'N/A'
        }])
        
        # Save to Excel with multiple sheets
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Optimization_Results', index=False)
            termination_info.to_excel(writer, sheet_name='Termination_Info', index=False)
        
        print(f"\n Results saved to: {output_path}")
        print(f" - Optimization results: 'Optimization_Results' sheet")
        print(f" - Termination information: 'Termination_Info' sheet")
        
        return output_path
    
    def plot_convergence(self, save_path: str = None):
        """Plot convergence curve with constraint information and early termination indicator"""
        if save_path is None:
            save_path = os.path.join(self.base_path, 'convergence.jpg')
            
        iterations = range(1, len(self.best_values_history) + 1)
        
        plt.rcParams['font.family'] = 'Arial'
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Plot 1: Convergence curve
        ax1.plot(iterations, self.best_values_history,
                label='Combined Objective', color='darkblue', linestyle='-',
                marker='o', markersize=4, markerfacecolor='#2A7CCC', 
                markeredgecolor='black', linewidth=2)
        
        ax1.plot(iterations, self.constraint_penalty_history,
                label='Constraint Penalty', color='red', linestyle='--',
                marker='s', markersize=4, markerfacecolor='red', 
                markeredgecolor='black', linewidth=2)
        
        # Add early termination indicator
        if self.early_terminated:
            ax1.axvline(x=len(self.best_values_history), color='orange', linestyle=':', 
                       linewidth=3, label=f'Early Termination (Iter {len(self.best_values_history)})')
            ax1.text(len(self.best_values_history), max(self.best_values_history) * 0.8, 
                    'Early\nTermination', rotation=90, ha='center', va='bottom',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="orange", alpha=0.7))
        
        ax1.set_xlabel('Iteration', fontdict={'fontsize': 12, 'fontname': 'Arial'})
        ax1.set_ylabel('Objective Value', fontdict={'fontsize': 12, 'fontname': 'Arial'})
        ax1.set_title('MOPSO Convergence with Constraints and Early Termination', fontsize=14, fontweight='bold')
        ax1.legend(prop={'family': 'Arial', 'size': 10})
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Feasibility tracking
        feasible_points = [1 if penalty == 0.0 else 0 for penalty in self.constraint_penalty_history]
        ax2.fill_between(iterations, feasible_points, alpha=0.3, color='green', label='Feasible Region')
        ax2.plot(iterations, feasible_points, color='darkgreen', linewidth=2, label='Feasibility')
        
        # Add early termination indicator to second plot
        if self.early_terminated:
            ax2.axvline(x=len(self.best_values_history), color='orange', linestyle=':', 
                       linewidth=3, label=f'Early Termination (Iter {len(self.best_values_history)})')
        
        ax2.set_xlabel('Iteration', fontdict={'fontsize': 12, 'fontname': 'Arial'})
        ax2.set_ylabel('Feasible (1) / Infeasible (0)', fontdict={'fontsize': 12, 'fontname': 'Arial'})
        ax2.set_title('Solution Feasibility Over Time', fontsize=14, fontweight='bold')
        ax2.legend(prop={'family': 'Arial', 'size': 10})
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim(-0.1, 1.1)
        
        # Add termination information as text
        termination_text = f"Termination: {self.termination_reason}"
        if len(termination_text) > 60:
            termination_text = termination_text[:60] + "..."
        
        plt.figtext(0.02, 0.02, termination_text, fontsize=10, 
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
        
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.1)  # Make room for termination text
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        print(f" Convergence plot saved to: {save_path}")
        plt.show()

    def plot_actual_objective_histories(self, save_path: str = None):
        """
        Plot actual (unscaled, unweighted) value histories for all objectives.
        """
        if save_path is None:
            save_path = os.path.join(self.base_path, 'actual_objective_histories.jpg')
        
        iterations = range(1, len(self.actual_value_history) + 1)
        obj_names = list(self.objective_targets.keys())
        actual_values = np.array(self.actual_value_history)  # shape: (iterations, num_objs)

        plt.rcParams['font.family'] = 'Arial'
        fig, axes = plt.subplots(len(obj_names), 1, figsize=(12, 4 * len(obj_names)), sharex=True)

        if len(obj_names) == 1:
            axes = [axes]  # make iterable for consistency

        # Plot each objective history
        for i, obj_name in enumerate(obj_names):
            axes[i].plot(
                iterations, actual_values[:, i],
                label=f'Actual {obj_name}', color='blue',
                marker='o', markersize=4, linewidth=2
            )
        
            # Early termination marker
            if self.early_terminated:
                axes[i].axvline(
                    x=len(self.best_values_history), color='orange', linestyle=':',
                    linewidth=2, label=f'Early Termination (Iter {len(self.best_values_history)})'
                )
        
            axes[i].set_ylabel(f'{obj_name} Value', fontdict={'fontsize': 12, 'fontname': 'Arial'})
            axes[i].set_title(f'History of {obj_name}', fontsize=14, fontweight='bold')
            axes[i].legend(prop={'family': 'Arial', 'size': 10})
            axes[i].grid(False)

        axes[-1].set_xlabel('Iteration', fontdict={'fontsize': 12, 'fontname': 'Arial'})

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f" Actual objective histories plot saved to: {save_path}")
        plt.show()
    
    def print_final_results(self, best_solution: Dict[str, Any], best_value: float):
        """Print final optimization results with constraint information and early termination details"""
        print("\n" + "="*60)
        print("FINAL OPTIMIZATION RESULTS")
        print("="*60)
        
        # Print termination information
        print(f"\nTermination Information:")
        print(f"  Status: {'Early Terminated' if self.early_terminated else 'Completed All Iterations'}")
        print(f"  Reason: {self.termination_reason}")
        print(f"  Total iterations: {len(self.best_values_history)}")
        if self.early_terminated:
            print(f"  Convergence threshold: {self.convergence_threshold}")
            print(f"  Stagnation limit: {self.stagnation_limit} iterations")
            if self.convergence_window:
                print(f"  Final convergence window: {[f'{v:.6f}' for v in self.convergence_window]}")
                print(f"  Window variation: {abs(max(self.convergence_window) - min(self.convergence_window)):.6f}")
        
        print("\nBest Parameter Values:")
        for param_name, value in best_solution.items():
            print(f"  {param_name}: {value}")
        
        print(f"\nBest Combined Objective Value: {best_value:.6f}")
        
        # Get final objective values
        param_names = list(self.parameter_info.keys())
        x_array = []
        for param_name in param_names:
            param_info = self.parameter_info[param_name]
            value = best_solution[param_name]
            
            if param_info['type'] == 'discrete':
                idx = param_info['values'].index(value)
                x_array.append(idx)
            else:
                x_array.append(float(value))
        
        final_objectives = self.objective_function(np.array(x_array))
        final_violations = self.check_constraint_violations(final_objectives)
        final_penalties, total_penalty = self.calculate_constraint_penalties(final_objectives)
        
        print("\nFinal Objective Values:")
        for i, obj_name in enumerate(self.objective_targets.keys()):
            print(f"  {obj_name}: {final_objectives[i]:.4f}")
        
        # Print constraint status
        print(f"\nConstraint Status:")
        print(f"  Total Penalty: {total_penalty:.6f}")
        print(f"  Solution is {'FEASIBLE' if total_penalty == 0.0 else 'INFEASIBLE'}")
        
        if total_penalty > 0.0:
            print("\n  Constraint Violations:")
            for obj_name, violation_info in final_violations.items():
                if violation_info['has_violation']:
                    constraint_config = self.objective_constraints[obj_name]
                    print(f"    {obj_name}:")
                    
                    if violation_info['min_violation'] is not None:
                        min_limit = constraint_config['min_constraint']
                        actual_val = final_objectives[list(self.objective_targets.keys()).index(obj_name)]
                        print(f"      Minimum violation: {actual_val:.4f} < {min_limit:.4f} (violation: {violation_info['min_violation']:.4f})")
                    
                    if violation_info['max_violation'] is not None:
                        max_limit = constraint_config['max_constraint']
                        actual_val = final_objectives[list(self.objective_targets.keys()).index(obj_name)]
                        print(f"      Maximum violation: {actual_val:.4f} > {max_limit:.4f} (violation: {violation_info['max_violation']:.4f})")
        
        # Print constraint summary
        print(f"\nConstraint Summary:")
        for obj_name in self.objective_targets.keys():
            constraint_config = self.objective_constraints[obj_name]
            if constraint_config['has_constraints']:
                print(f"  {obj_name}:")
                if constraint_config['min_constraint'] is not None:
                    print(f"    Minimum: >= {constraint_config['min_constraint']}")
                if constraint_config['max_constraint'] is not None:
                    print(f"    Maximum: <= {constraint_config['max_constraint']}")
                print(f"    Penalty Weight: {constraint_config['penalty_weight']}")
            else:
                print(f"  {obj_name}: No constraints")
        
        # Statistics about feasible solutions
        feasible_solutions = sum(1 for penalty in self.constraint_penalty_history if penalty == 0.0)
        total_solutions = len(self.constraint_penalty_history)
        feasibility_rate = (feasible_solutions / total_solutions) * 100
        
        print(f"\nOptimization Statistics:")
        print(f"  Total iterations: {total_solutions}")
        print(f"  Feasible solutions: {feasible_solutions} ({feasibility_rate:.1f}%)")
        print(f"  Infeasible solutions: {total_solutions - feasible_solutions} ({100-feasibility_rate:.1f}%)")
        
        # Early termination efficiency statistics
        if self.early_terminated:
            max_possible_iter = 3000  # You can make this a parameter
            efficiency_gain = ((max_possible_iter - total_solutions) / max_possible_iter) * 100
            print(f"  Early termination efficiency: {efficiency_gain:.1f}% iterations saved")
        
        if feasible_solutions > 0:
            print(f"\n Found {feasible_solutions} feasible solution(s) during optimization")
        else:
            print(f"\n  No feasible solutions found. Consider:")
            print(f"     - Relaxing constraints")
            print(f"     - Increasing penalty weights")
            print(f"     - Expanding parameter bounds")
            print(f"     - Increasing number of iterations")
            if self.early_terminated:
                print(f"     - Adjusting convergence threshold (current: {self.convergence_threshold})")
                print(f"     - Increasing stagnation limit (current: {self.stagnation_limit})")


# Custom exception for early termination
class EarlyTerminationException(Exception):
    """Custom exception to handle early termination in PSO optimization"""
    pass


# Main execution
if __name__ == "__main__":
    # Create optimizer instance
    optimizer = GeneralizedMOPSO()
    
    try:
        # Setup environment
        optimizer.setup_environment()
        
        # Get optimization parameters
        print("\n--- OPTIMIZATION PARAMETERS ---")
        swarmsize = int(input("Enter swarm size (default 20): ") or "20")
        maxiter = int(input("Enter maximum iterations (default 3000): ") or "3000")
        
        # Get early termination parameters
        print("\n--- EARLY TERMINATION PARAMETERS ---")
        use_default_termination = input("Use default early termination settings? (y/n, default: y): ").strip().lower()
        
        if use_default_termination != 'n':
            print("Using default settings:")
            print("  - Convergence threshold: 10e-6")
            print("  - Stagnation limit: 50 iterations")
        else:
            custom_threshold = input("Enter convergence threshold (default: 10e-6): ").strip()
            if custom_threshold:
                optimizer.convergence_threshold = float(custom_threshold)
            
            custom_stagnation = input("Enter stagnation limit in iterations (default: 50): ").strip()
            if custom_stagnation:
                optimizer.stagnation_limit = int(custom_stagnation)
        
        print(f"\nFinal early termination settings:")
        print(f"  - Convergence threshold: {optimizer.convergence_threshold}")
        print(f"  - Stagnation limit: {optimizer.stagnation_limit} iterations")
        print(f"  - Termination occurs when predicted values show no significant")
        print(f"    variation (< {optimizer.convergence_threshold}) for {optimizer.stagnation_limit} consecutive iterations")
        
        # Run optimization
        best_solution, best_value = optimizer.run_optimization(swarmsize, maxiter)
        
        # Save results
        optimizer.save_results()
        
        # Plot convergence
        optimizer.plot_convergence()
        optimizer.plot_actual_objective_histories()
        
        # Print final results
        optimizer.print_final_results(best_solution, best_value)
        
    except Exception as e:
        print(f"\n Error during optimization: {e}")
        raise